In [1]:
import quandl
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostRegressor        #AdaBoostClassifier,RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split,RandomizedSearchCV #Hiperparametrización
quandl.ApiConfig.api_key = 'DEN8cmx6MYrtzyeUdDeh' #????

In [2]:
%%time
tickers = ['FB','GOOGL','AAPl','AMZN','TSLA','MSFT','AMD','INTC','T','NFLX']
Validate = [] #Conjunto para validación 
Train = [] #Conjunto de entrenamiento

for ticker in tickers:
    df = quandl.get("WIKI/"+ticker)
    #print (df)
    df = df.iloc[-500:]
    func = ['mean','std','skew','kurt'] #Estadísticos de interés
    time = [2,4,7,30,60,120,250] #Días tomados artibatriamente  
    Z = pd.DataFrame()  
    for f in func: 
        for t in time:
            aux = pd.DataFrame()
            for i in range(249):
                aux[ df.index[i+251] ] = list( df.iloc[i-t+251:i+251]['Adj. Close'] ) #
            Z[f+str(t)] = aux.apply(f,axis=0)  
    Z.drop(['skew2','kurt2'],axis=1, inplace =True)
    W = pd.merge(Z, df[['Adj. Close']],left_index=True,right_index=True)    
    X = W.drop(['Adj. Close'],1).fillna(0)
    y = W[['Adj. Close']]    
    mm1 = MinMaxScaler()
    mm2 = MinMaxScaler()
    mm1.fit(X)
    mm2.fit(y)
    Xm = pd.DataFrame(mm1.transform(X))
    ym = pd.DataFrame(mm2.transform(y))
    Xt, Xv, yt, yv = train_test_split(Xm, ym, test_size = 0.3)
    model = AdaBoostRegressor()
    model.fit(Xt,yt)   
    print ticker
    print 'ACC Validate {:.2%}'.format(model.score(Xv,yv))
    print 'ACC Train    {:.2%}'.format(model.score(Xt,yt))
    Validate.append(model.score(Xv,yv))
    Train.append(model.score(Xt,yt))
print '\n'
print 'Promedio Validate  {:.2%}'.format(np.array(Validate).mean())
print 'Promedio Train     {:.2%}'.format(np.array(Train).mean())

/home/ciprian/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


FB
ACC Validate 96.22%
ACC Train    98.29%
GOOGL
ACC Validate 95.91%
ACC Train    98.35%
AAPl
ACC Validate 95.24%
ACC Train    98.15%
AMZN
ACC Validate 99.08%
ACC Train    99.55%
TSLA
ACC Validate 77.34%
ACC Train    93.58%
MSFT
ACC Validate 98.02%
ACC Train    99.37%
AMD
ACC Validate 89.35%
ACC Train    91.75%
INTC
ACC Validate 97.22%
ACC Train    99.30%
T
ACC Validate 89.29%
ACC Train    93.87%
NFLX
ACC Validate 98.20%
ACC Train    99.40%


Promedio Validate  93.59%
Promedio Train     97.16%
CPU times: user 2min 54s, sys: 1.21 s, total: 2min 55s
Wall time: 3min 19s


Observación: Los datos que estaban dando como Target son continuos entonces se debía cambiar la SVM en modo regresión 